In [ ]:
import numpy as np
import pandas as pd
import healpy as hp
import matplotlib
from matplotlib import pyplot as plt

In [ ]:
#Change this for each of the filters
df = pd.read_csv('Z087_files_data.csv')

In [ ]:
def get_vertices(df, index):
    '''
    Function to convert ra and dec values into vectors compatible with healpy
    Inputs: The dataframe and the index value of the row corresponding to the SCA that will be plotted.
    Outputs: An array of vertices in the form of vectors for each corner of the SCA.
    '''
    ra1 = df['RA1'][index]
    ra2 = df['RA2'][index]
    ra3 = df['RA3'][index]
    ra4 = df['RA4'][index]
    dec1 = df['DEC1'][index]
    dec2 = df['DEC2'][index]
    dec3 = df['DEC3'][index]
    dec4 = df['DEC4'][index]
    def ra_dec_to_theta_phi(ra, dec):
        ra_rad = np.radians(ra)
        dec_rad = np.radians(dec)
        theta = np.pi / 2 - dec_rad
        phi = ra_rad
        return theta, phi
    theta1, phi1 = ra_dec_to_theta_phi(ra1, dec1)
    theta2, phi2 = ra_dec_to_theta_phi(ra2, dec2)
    theta3, phi3 = ra_dec_to_theta_phi(ra3, dec3)
    theta4, phi4 = ra_dec_to_theta_phi(ra4, dec4)
    vec1 = hp.ang2vec(theta1, phi1)
    vec2 = hp.ang2vec(theta2, phi2)
    vec3 = hp.ang2vec(theta3, phi3)
    vec4 = hp.ang2vec(theta4, phi4)
    vertices = np.array([vec1, vec2, vec3, vec4])
    return vertices

In [ ]:
obs_rows = df.index

NSIDE = 4096
NPIX  = hp.nside2npix(NSIDE)
ipix_box_list = []
for index in obs_rows:
    vertices = get_vertices(index)
    ipix_box_single = hp.query_polygon(nside=NSIDE, vertices=vertices, inclusive=False)
    ipix_box_list.append(ipix_box_single)
ipix_box = np.concatenate(ipix_box_list)
rot = [9.7, -44]
m = np.zeros(hp.nside2npix(NSIDE))
counts = np.bincount(ipix_box, minlength=len(m))
m[:len(counts)] = counts
#get only the 2-d array for the projected map
array = hp.gnomview(m, rot=rot,title="Sky Location of Roman Pictures-Z087 Band", reso = 1.6, xsize = 200, return_projected_map = True, no_plot=True)

In [ ]:
diagonal = np.array([]) #get the diagonal entries of the 2-d array
for i in range(200):
    entry = array[i, i]
    diagonal = np.append(entry, diagonal)

In [ ]:
diagonal = diagonal[diagonal != 0] #drop the zeros to just include where the exposure map has SCAs

In [ ]:
#RA1 is on the top right corner of an SCA and RA4 is on the top left corner of an SCA. In the gnomview, increasing goes from right to left.

In [ ]:
ra1s = df['RA1'].values

In [ ]:
ra4s = df['RA4'].values

In [ ]:
ramin = np.min(ra1s)

In [ ]:
ramax = np.max(ra4s)

In [ ]:
xdata = np.linspace(ramin, ramax, 139) #range of ra values for the entire exposure map

In [ ]:
centers = df['MAJOR ROT AXS- RA'].values #center of the exposure map
center_ra = centers[0]
center_ra

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(xdata, diagonal)
plt.axvline(x=center_ra, color='r', linestyle='--', label='Center of Exposure Map') #plot a vertical line to show the ra value of the center of the exposure map.
plt.title('Distribution of SCAs')
plt.xlabel('RA')
plt.ylabel('Number of SCAs')
plt.legend()
plt.grid(True)
plt.show()